In [1]:
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.autograd import Variable

import torch.backends.cudnn as cudnn
cudnn.benchmark = True

In [2]:
train_feature = np.load('train_feature.npy')
train_label = np.load('train_label.npy')
test_feature = np.load('test_feature.npy')
test_label = np.load('test_label.npy')

In [3]:
hidden_size = 64
num_layers = 1
learning_rate = 0.01
EPOCH = 600

In [4]:
train_feature = torch.from_numpy(train_feature)
train_feature = train_feature.float()
train_feature = Variable(train_feature)
train_feature = train_feature.cuda()

train_label = torch.from_numpy(train_label)
train_label = train_label.float()
train_label = Variable(train_label)
train_label = train_label.cuda()

test_feature = torch.from_numpy(test_feature)
test_feature = test_feature.float()
test_feature = Variable(test_feature)
test_feature = test_feature.cuda()

In [5]:
class LSTMRG(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=1, num_layers=2):
        super(LSTMRG, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers)
        self.reg = nn.Linear(hidden_size, output_size)  # 拼接全连接层

    def forward(self, x):
        x, _ = self.rnn(x)
        s, b, h = x.shape  # (seq, batch, hidden_size)
        x = x.view(s * b, h)
        x = self.reg(x)
        x = x.view(s, b, -1)
        return x

In [6]:
for istation in range(81):
    for jinout in range(2):
        net = LSTMRG(input_size = train_feature.shape[-1], hidden_size = hidden_size, output_size=1, num_layers=num_layers)
        net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
        net.cuda()
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
        for epoch in range(EPOCH):
            net.train()
            out = net(train_feature[istation][jinout])
            loss = criterion(out, train_label[istation][jinout])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (epoch + 1) % EPOCH == 0:
                net.eval()
                pred_label = net(test_feature[istation][jinout])
                pred_label = pred_label.data.cpu().numpy()
                mse = np.mean((pred_label - test_label[istation][jinout]) ** 2)
                print('\tTrain Epoch: %d, train_loss: %.7f, test_loss: %.7f' % (epoch + 1, loss, mse))
        torch.save(net.state_dict(), 'model/station-' + str(istation).zfill(2) + '-' + str(jinout) + '.pth')

	Train Epoch: 600, train_loss: 0.0010536, test_loss: 0.0014383
	Train Epoch: 600, train_loss: 0.0020553, test_loss: 0.0039976
	Train Epoch: 600, train_loss: 0.0003715, test_loss: 0.0005594
	Train Epoch: 600, train_loss: 0.0014555, test_loss: 0.0024732
	Train Epoch: 600, train_loss: 0.0010827, test_loss: 0.0015273
	Train Epoch: 600, train_loss: 0.0005349, test_loss: 0.0006747
	Train Epoch: 600, train_loss: 0.0006395, test_loss: 0.0007507
	Train Epoch: 600, train_loss: 0.0014343, test_loss: 0.0027720
	Train Epoch: 600, train_loss: 0.0008226, test_loss: 0.0068344
	Train Epoch: 600, train_loss: 0.0003835, test_loss: 0.0003890
	Train Epoch: 600, train_loss: 0.0016433, test_loss: 0.0028040
	Train Epoch: 600, train_loss: 0.0004031, test_loss: 0.0004841
	Train Epoch: 600, train_loss: 0.0015404, test_loss: 0.0035509
	Train Epoch: 600, train_loss: 0.0029878, test_loss: 0.0078303
	Train Epoch: 600, train_loss: 0.0034660, test_loss: 0.0044373
	Train Epoch: 600, train_loss: 0.0012569, test_loss: 0.